In [1]:
##import the required libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv1D,BatchNormalization,Dense,Dropout,Embedding,Flatten,MaxPool1D
from keras.layers import LeakyReLU,PReLU
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

/home/surya/DL/lib/python3.5/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)
Using TensorFlow backend.


### Data preparation

In [2]:
train = pd.read_csv("c771afa0-c-HackerEarthML4Updated/train_data.csv")
test = pd.read_csv("c771afa0-c-HackerEarthML4Updated/test_data.csv")

In [32]:
x_test = test.drop(['connection_id'],axis=1)
X_t = x_test.as_matrix()

In [3]:
train.head()

,connection_id,cont_1,cont_2,cont_3,cont_4,cont_5,cont_6,cont_7,cont_8,cont_9,...,cat_15,cat_16,cat_17,cat_18,cat_19,cat_20,cat_21,cat_22,cat_23,target
0,cxcon_1,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,2
1,cxcon_4,0,520,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
2,cxcon_7,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
3,cxcon_10,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
4,cxcon_13,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,2


In [4]:
x_train = train.drop(['connection_id','target'],axis=1)
y_train = train['target']

In [5]:
x_train.columns

Index(['cont_1', 'cont_2', 'cont_3', 'cont_4', 'cont_5', 'cont_6', 'cont_7',
       'cont_8', 'cont_9', 'cont_10', 'cont_11', 'cont_12', 'cont_13',
       'cont_14', 'cont_15', 'cont_16', 'cont_17', 'cont_18', 'cat_1', 'cat_2',
       'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9', 'cat_10',
       'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17',
       'cat_18', 'cat_19', 'cat_20', 'cat_21', 'cat_22', 'cat_23'],
      dtype='object')

In [6]:
xtrain,xtest,ytrain,ytest = train_test_split(x_train,y_train,test_size=0.3)

In [7]:
xtrain,xtest,ytrain,ytest = xtrain.as_matrix(),xtest.as_matrix(),ytrain.as_matrix(),ytest.as_matrix()

In [8]:

xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

((118514, 41), (50793, 41), (118514,), (50793,))

In [9]:
y_cat,y_cat_test = to_categorical(ytrain),to_categorical(ytest)

In [ ]:
m = max([max([i for i in x]) for x in xtrain])
m

In [ ]:
len([[i for i in x if i > 1000000] for x in xtrain if ])

In [17]:
mean_px = xtrain.mean().astype(np.float32)
std_px = xtrain.std().astype(np.float32)

In [18]:
def norm_input(x): return (x-mean_px)/std_px

### Model using Neural nets.

In [13]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Lambda(norm_input, input_shape=(1,28,28)))
    model.add(Flatten())
    model.add(Dense(100, input_dim=41, activation='sigmoid'))
    model.add(LeakyReLU(0.3))
    model.add(BatchNormalization())
    model.add(Dense(100, input_dim=41, activation='sigmoid'))
    model.add(Dense(100, input_dim=41, activation='sigmoid'))    
    model.add(Dense(100,input_dim=1000,activation='sigmoid'))
    model.add(PReLU(alpha_initializer='zeros'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [64]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               4200      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_9 (Dense)              (None, 100)               10100     
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 100)               100       
__________

In [14]:
model = baseline_model()

In [21]:
model.optimizer.lr = 0.01

### Fit the Model with train and cross validate data

In [22]:
model.fit(xtrain,y_cat,epochs=5,batch_size=32,validation_data=([xtest,y_cat_test]))

Train on 118514 samples, validate on 50793 samples
Epoch 1/5
118514/118514 [==============================] - 43s 359us/step - loss: 0.4882 - acc: 0.7778 - val_loss: 0.4790 - val_acc: 0.7818
Epoch 2/5
118514/118514 [==============================] - 44s 368us/step - loss: 0.4870 - acc: 0.7781 - val_loss: 0.4810 - val_acc: 0.7817
Epoch 3/5
118514/118514 [==============================] - 41s 348us/step - loss: 0.4880 - acc: 0.7782 - val_loss: 0.4815 - val_acc: 0.7820
Epoch 4/5
118514/118514 [==============================] - 47s 394us/step - loss: 0.4881 - acc: 0.7779 - val_loss: 0.4804 - val_acc: 0.7816
Epoch 5/5
118514/118514 [==============================] - 42s 352us/step - loss: 0.4888 - acc: 0.7777 - val_loss: 0.4813 - val_acc: 0.7817


In [26]:
X = x_train.as_matrix()
y = y_train.as_matrix()

In [ ]:
y_cat_idx = {0:[1,0,0],
            1:[0,1,0],
            2:[0,0,1]}

In [28]:
ycat = to_categorical(y)

### Fit the data with whole training data

In [29]:
model.fit(X,ycat,epochs=5,batch_size=32)

Epoch 1/5
169307/169307 [==============================] - 51s 302us/step - loss: 0.4885 - acc: 0.7788
Epoch 2/5
169307/169307 [==============================] - 56s 331us/step - loss: 0.4893 - acc: 0.7784
Epoch 3/5
169307/169307 [==============================] - 53s 314us/step - loss: 0.4896 - acc: 0.7784
Epoch 4/5
169307/169307 [==============================] - 54s 321us/step - loss: 0.4885 - acc: 0.7790
Epoch 5/5
169307/169307 [==============================] - 57s 336us/step - loss: 0.4889 - acc: 0.7788


### Predict for the test data and prepare submission file

In [ ]:
pred = model.predict(X_t)

In [48]:
pred_data = []
for i in pred:
    pred_data.append(np.argmax(i))


[1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,


In [58]:
test['target'] = pred_data

In [59]:
df = test[['connection_id','target']]

In [63]:
df.to_csv("submission.csv",index=False)